In [1]:
import os
import pandas as pd
import numpy as np
# import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from graph import GNNNodeClassifier
import utils


2023-05-11 22:33:08.276838: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 22:33:09.251857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def run_experiment(model, x_train, y_train):
    # Compile the model.
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    )
    # Create an early stopping callback.
    early_stopping = keras.callbacks.EarlyStopping(
        monitor="val_acc", patience=50, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_split=0.15,
        callbacks=[early_stopping],
    )

    return history



In [3]:
def get_graph_info():
    # # Create an edges array (sparse adjacency matrix) of shape [2, num_edges].
    # edges = citations[["source", "target"]].to_numpy().T
    # # Create an edge weights array of ones.
    # edge_weights = tf.ones(shape=edges.shape[1])
    # # Create a node features array of shape [num_nodes, num_features].
    # node_features = tf.cast(
    #     papers.sort_values("paper_id")[feature_names].to_numpy(), dtype=tf.dtypes.float32
    # )
    # # Create graph info tuple with node_features, edges, and edge_weights.
    # graph_info = (node_features, edges, edge_weights)
    #
    # print("Edges shape:", edges.shape)
    # print("Nodes shape:", node_features.shape)
    return ("node_features", "edges", "edge_weights")


In [4]:
def getStatus(statusDesc) -> int:
    return statusDesc == "Accepted" if 1 else 0


In [5]:
hidden_units = [32, 32]
learning_rate = 0.01
dropout_rate = 0.5
num_epochs = 300
batch_size = 256

problem_id = "p03949"
problem_data = pd.read_csv("./data/Project_CodeNet/metadata/p03949.csv")
language_extension = ("C++", "cpp")



In [6]:
data = {"submission_id": [], "code": []}

for row in range(len(problem_data)):
    submission_id = problem_data.iloc[[row]]["submission_id"].values
    path = f'./data/Project_CodeNet/data/{problem_id}/{language_extension[0]}/{submission_id[0]}.{language_extension[1]}'
    if os.path.exists(path):
        with open(path, encoding="utf8") as f:
            code = f.readlines()
            # print(submission_id, code)
            data["submission_id"].append(submission_id[0])
            data["code"].append(code[0])



In [7]:
code_df = pd.DataFrame(data)
code_df=code_df.set_index('submission_id')
problem_data=problem_data.set_index('submission_id')
problem_data=problem_data.join(code_df, on='submission_id')



In [9]:
problem_data

,problem_id,user_id,date,language,original_language,filename_ext,status,cpu_time,memory,code_size,accuracy,code
submission_id,,,,,,,,,,,,
s182099834,p03949,u846379833,1601312949,C++,C++ (GCC 9.2.1),cpp,Accepted,254.0,18420.0,1992,NaN,#include <bits/stdc++.h>\n
s904840258,p03949,u846379833,1601312515,C++,C++ (GCC 9.2.1),cpp,Wrong Answer,244.0,18388.0,2002,NaN,#include <bits/stdc++.h>\n
s262047390,p03949,u863370423,1601000823,C++,C++ (GCC 9.2.1),cpp,Accepted,62.0,15076.0,1701,NaN,#include<bits/stdc++.h>\n
s673153461,p03949,u816631826,1601000583,C++,C++ (GCC 9.2.1),cpp,Wrong Answer,68.0,14604.0,1759,NaN,#include<bits/stdc++.h>\n
s728796387,p03949,u513211419,1600965129,C++,C++ (GCC 9.2.1),cpp,Accepted,226.0,27964.0,21384,NaN,/*\tauthor: Kite_kuma\n
...,...,...,...,...,...,...,...,...,...,...,...,...
s273733813,p03949,u354336886,1478485345,C++,C++14 (GCC 5.4.1),cpp,Accepted,81.0,21248.0,2319,NaN,#include <bits/stdc++.h>\n
s653192894,p03949,u207056619,1478485340,C++,C++14 (GCC 5.4.1),cpp,Wrong Answer,569.0,23168.0,1910,NaN,#include <iostream>\n
s117052253,p03949,u280354208,1478485266,C++,C++14 (GCC 5.4.1),cpp,Wrong Answer,105.0,13184.0,3896,NaN,// {{{ by shik\n


In [10]:
problem_data=problem_data[['status','code']]
problem_data

,status,code
submission_id,,
s182099834,Accepted,#include <bits/stdc++.h>\n
s904840258,Wrong Answer,#include <bits/stdc++.h>\n
s262047390,Accepted,#include<bits/stdc++.h>\n
s673153461,Wrong Answer,#include<bits/stdc++.h>\n
s728796387,Accepted,/*\tauthor: Kite_kuma\n
...,...,...
s273733813,Accepted,#include <bits/stdc++.h>\n
s653192894,Wrong Answer,#include <iostream>\n
s117052253,Wrong Answer,// {{{ by shik\n


In [11]:
# split data
train_data, test_data = [], []

for _, group_data in problem_data.groupby("submission_id"):
    # Select around 50% of the dataset for training.
    random_selection = np.random.rand(len(group_data.index)) <= 0.5
    train_data.append(group_data[random_selection])
    test_data.append(group_data[~random_selection])

train_data = pd.concat(train_data).sample(frac=1)
test_data = pd.concat(test_data).sample(frac=1)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)



Train data shape: (1895, 2)
Test data shape: (1915, 2)


In [14]:
train_data['status']

submission_id
s865993703        Accepted
s780160826    Wrong Answer
s349694722        Accepted
s479803191    Wrong Answer
s626857453    Wrong Answer
                  ...     
s514220718    Wrong Answer
s408565079    Wrong Answer
s661477777        Accepted
s655641161        Accepted
s644934678        Accepted
Name: status, Length: 1895, dtype: object

In [15]:
# Create train and test features as a numpy array.
x_train = train_data["code"].to_numpy()
x_test = test_data["code"].to_numpy()
# Create train and test targets as a numpy array.
y_train = train_data["status"]
y_test = test_data["status"]

model = GNNNodeClassifier(get_graph_info(), 2, hidden_units)
run_experiment(model, x_train, y_train)
# feature_names = set(problem_data.columns) - {"paper_id", "subject"}
# num_features = len(feature_names)
# num_classes = len(class_idx)



2023-05-11 22:36:29.551788: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:36:29.680683: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


InvalidArgumentError: Value for attr 'T' of string is not in the list of allowed values: float, double, int32, uint8, int16, int8, complex64, int64, qint8, quint8, qint32, bfloat16, qint16, quint16, uint16, complex128, half, uint32, uint64
	; NodeDef: {{node Sum}}; Op<name=Sum; signature=input:T, reduction_indices:Tidx -> output:T; attr=keep_dims:bool,default=false; attr=T:type,allowed=[DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, 10440210506161272279, DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]; attr=Tidx:type,default=DT_INT32,allowed=[DT_INT32, DT_INT64]> [Op:Sum]